In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
  temperature=0.1, 
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
  ]
)

In [2]:
#랭체인에는 5갖 정도의 메모리가 있는데, 모드 저장 방식도 다르고, 각자만의 장단점이 있다.
#챗복에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없고, 대화를 이해할 수 있는 능력이 없다.
#openai 에서 default 로 제공하는 것은 메모리를 지원하지 않아 stateless 하다.
#모델에게 무언가를 보내면 응답한 후 모든 대화 내용을 저장하지 않아 잊는다.
#chatGPT에는 메모리가 탑재되어있어 실제로 어떤 사람과 이야기하고 있다는 느낌을 들게 한다.

#Conversation Buffer memory - 단순히 이전 대화 내용 전체를 저장하는 메모리
#단점은 대화 내용이 길어질수록 메모리도 계속 커지기 때문에 비효율적이다.
#모델 자체에는 메모리가 없기 때문에 모델에 요청을 보낼 때 이전 대화 기록 전체를 같이 보내야한다.
#대화가 길어질수록 메모리에 보내야될 대화가 길어진다.

from langchain.memory import ConversationBufferMemory

#text completion을 할때 유용하다.예측을하고, 텍스트를 자동 완성 하고 싶을 때
#챗모델과 작업을 하려면 스트링이 아닌 ai 메세지와 human메세지가 모두 필요하다.
#메모리 종류와 무관하게 api는 다 같다.(return_message, save_context, load_memory...등 을 갖고 있음)
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input":"hi"},{"output":"how are you"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'), AIMessage(content='how are you')]}

In [4]:
#대화의 특정 부분만을 저장하는 메모리
#가장 최근의 5개의 메세지를 저장한다고 했을 떄, 6번째 메세지가 추가되었을 때 가장 오래된 메세지는 버려지는 방식
#메모리를 특정 크기로 유지할 수 있는것이 장점
#단잠은 챗봇이 전체 대화가 아닌 최근 대화에만 집중한다.
from langchain.memory import ConversationBufferWindowMemory

#k-how many interaction you want to remember
memory = ConversationBufferWindowMemory(
  return_messages=True, 
  k=4
)

def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})


add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [5]:
add_message(5,5)

# 첫번째는 사라짐
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}